# Midterm - Spring 2019

There are going to be three steps to this midterm assignment, based on something called the MAGGIC Risk Score found here: https://www.mdcalc.com/maggic-risk-calculator-heart-failure#evidence

The MAGGIC Risk Score is a way of measuring how likely a patient is to have heart failure.

For the midterm, you will need to do three things:

1. Create JSON in a file or series of files the encapsulate the calculation rules in the MAGGIC risk calculator.
2. Create a series of functions to calculate each component score (seven functions in total).
3. Create a function that calculates the MAGGIC score for a particular person.


# #1 Create JSON for MAGGIC calculation rules

In this directory, create one or more JSON files that you will use in steps 2 and 3 to calculate the MAGGIC scores.  This JSON you create will need to encapsulate the scoring rules from teh MAGGIC risk calculator so that, if the rules were to change slightly in the future or be different for different populations, then you could easily swap out a different set of rules.

You can find all of the rules here: https://www.mdcalc.com/maggic-risk-calculator-heart-failure#evidence

For example, when a MAGGIC scoring rule says that being `male` earns the patient and extra `+1`, you could encode that in JSON as a simple dictionary item:

```python
{
    "male": 1
}
```

Or when MAGGIC says that a patient's BMI range should give them additional points, you could encode that by storing a list of min, max, and points.  In my example, I'm going to assume the "max" is exclusive (i.e. min <= value < max).  If you were to put all of this in one file together with other rules, you might include this in a larger dictionary as shown below.  If it is a file by itself, you could just have the list for bmi in a stand-alone file.

```python
{
    "risk_factors": {
        "male": 1,
        "smoker": 1,
        "diabetic": 3
    },
    "bmi": [
        { "min":  0, "max": 15, "points": 6 },
        { "min": 15, "max": 20, "points": 5 },
        { "min": 20, "max": 25, "points": 3 },
        { "min": 25, "max": 30, "points": 2 }
    ]
}
```


There is a file `maggic.json` in this directory with the contents above that you can use as a sample to start with.  Don't assume what's in the file is correct, though!


**NOTE:** Your code should be able to handle input that comes to it in all lowercase characters.  Look in the testing cells for part 2 and 3 to see how your code will be tested.

In [1]:
# Remove any contents here and leave this blank

# #2 MAGGIC Scoring Functions

Write individual functions for all of your rules that are encoded in the JSON from part 1.  I've created a series of tests that will verify your functions are working correctly.  In the spaces for solutions below, I've provided the function name and parameters.  Use those, but you'll need to fill in the function.

**NOTE:** Your code should be able to handle input that comes to it in all lowercase characters.  Look in the testing cells below to see how your code will be tested.


## #2A - Risk Factors

In [2]:
import json

def risk_factors_score(risks_list, config_file):
    """ list, str -> int
    risk_list: A list of strings that include various risk factors this subject DOES have
    config_file: The name of a configuration file to use for scoring these risk factors
    
    The function should return a score for this subject based on the risk factors in the risk_list
    and the scoring rules in the config_file.  For instance, if the subject is Male and Diabetic,
    the risk score would be 1 + 3 = 4, and the function should return 4.
    """
    
    score = 0
    
    ### BEGIN SOLUTION
    
    config = json.load(open(config_file))
    risk_factor_scores = config.get("risk_factors")
    
    for risk in risks_list:
        if risk in risk_factor_scores:
            score += risk_factor_scores.get(risk)
    
    ### END SOLUTION

    return score


In [3]:
# Put the name of your configuration file below
RISK_FACTORS_CONFIG_FILE = "maggic.json"

In [4]:
assert(risk_factors_score([],RISK_FACTORS_CONFIG_FILE) == 0)
assert(risk_factors_score(['male'],RISK_FACTORS_CONFIG_FILE) == 1)
assert(risk_factors_score(['smoker'],RISK_FACTORS_CONFIG_FILE) == 1)
assert(risk_factors_score(['diabetic'],RISK_FACTORS_CONFIG_FILE) == 3)
assert(risk_factors_score(['copd'],RISK_FACTORS_CONFIG_FILE) == 2)
assert(risk_factors_score(['heart failure first diagnosed >= 18 months ago'],RISK_FACTORS_CONFIG_FILE) == 2)
assert(risk_factors_score(['not on beta blocker'],RISK_FACTORS_CONFIG_FILE) == 3)
assert(risk_factors_score(['not on ace-i/arb'],RISK_FACTORS_CONFIG_FILE) == 1)
assert(risk_factors_score(['not on ace-i/arb', 'male'],RISK_FACTORS_CONFIG_FILE) == 2)
assert(risk_factors_score(['smoker','male','copd'],RISK_FACTORS_CONFIG_FILE) == 4)

## #2B Ejection Fraction

**NOTE:** The ejection fraction is a percentage that can't be greater than 100.  Your code and configuration model can assume the value will always be between 0 and 100.

In [5]:
import json

def ejection_fraction_score(ef, config_file):
    """ int, str -> int
    ef: The ejection fraction measurement as an integer value
    config_file: The name of a configuration file to use for scoring the ejection fraction
    
    The function should return a score for this subject based on where their actual ejection fraction
    falls in the scoring rules.
    """
    
    ef_score = 0
    
    ### BEGIN SOLUTION
    
    config = json.load(open(config_file))
    ef_rules = config.get("ejection_fraction")
    
    for rule in ef_rules:
        if ef >= rule.get('min') and ef < rule.get('max'):
            ef_score = rule.get('points')
    
    ### END SOLUTION

    return ef_score


In [6]:
# Put the name of your configuration file below
EF_CONFIG_FILE = "maggic.json"

In [7]:
assert(ejection_fraction_score(0,  EF_CONFIG_FILE)==7)
assert(ejection_fraction_score(15, EF_CONFIG_FILE)==7)
assert(ejection_fraction_score(20, EF_CONFIG_FILE)==6)
assert(ejection_fraction_score(21, EF_CONFIG_FILE)==6)
assert(ejection_fraction_score(25, EF_CONFIG_FILE)==5)
assert(ejection_fraction_score(29, EF_CONFIG_FILE)==5)
assert(ejection_fraction_score(32, EF_CONFIG_FILE)==3)
assert(ejection_fraction_score(35, EF_CONFIG_FILE)==2)
assert(ejection_fraction_score(36, EF_CONFIG_FILE)==2)
assert(ejection_fraction_score(40, EF_CONFIG_FILE)==0)
assert(ejection_fraction_score(80, EF_CONFIG_FILE)==0)
assert(ejection_fraction_score(100,EF_CONFIG_FILE)==0)

## #2C NYHA Class

**NOTE:** If your function is provided an invalid NYHA class, please return 0.

In [8]:
import json

def nhya_class_score(nhya, config_file):
    """ int, str -> int
    nyha: The NHYA class for this subject
    config_file: The name of a configuration file to use for scoring the NHYA class
    
    The function should return a score for this subject based on where their NHYA class.
    
    See https://www.heart.org/en/health-topics/heart-failure/what-is-heart-failure/classes-of-heart-failure 
    if you're curious about the NHYA classification system.
    """
    
    nhya_score = 0
    
    ### BEGIN SOLUTION
    
    config = json.load(open(config_file))
    nhya_rules = config.get("nhya_class")

    nhya_score = nhya_rules.get(str(nhya))
    if nhya_score == None:
        nhya_score = 0
    
    ### END SOLUTION

    return nhya_score


In [9]:
# Put the name of your configuration file below
NHYA_CONFIG_FILE = "maggic.json"

In [10]:
assert(nhya_class_score(0,NHYA_CONFIG_FILE)==0)
assert(nhya_class_score(1,NHYA_CONFIG_FILE)==0)
assert(nhya_class_score(2,NHYA_CONFIG_FILE)==2)
assert(nhya_class_score(3,NHYA_CONFIG_FILE)==6)
assert(nhya_class_score(4,NHYA_CONFIG_FILE)==8)
assert(nhya_class_score(5,NHYA_CONFIG_FILE)==0)

## #2D Creatinine Level

In [11]:
import json

def creatinine_level_score(creatinine, config_file):
    """ int, str -> int
    creatinine: The creatinine measurement as an integer value
    config_file: The name of a configuration file to use for scoring the creatinine level
    
    The function should return a score for this subject based on where their creatinine level
    falls in the scoring rules.
    """
    
    c_score = 0
    
    ### BEGIN SOLUTION
    
    config = json.load(open(config_file))
    c_rules = config.get("creatinine_level")
    
    for rule in c_rules:
        if creatinine >= rule.get('min') and creatinine < rule.get('max'):
            c_score = rule.get('points')
    
    ### END SOLUTION

    return c_score


In [12]:
# Put the name of your configuration file below
CREATININE_CONFIG_FILE = "maggic.json"

In [13]:
assert(creatinine_level_score(0,CREATININE_CONFIG_FILE)==0)
assert(creatinine_level_score(10,CREATININE_CONFIG_FILE)==0)
assert(creatinine_level_score(90,CREATININE_CONFIG_FILE)==1)
assert(creatinine_level_score(100,CREATININE_CONFIG_FILE)==1)
assert(creatinine_level_score(115,CREATININE_CONFIG_FILE)==2)
assert(creatinine_level_score(121,CREATININE_CONFIG_FILE)==2)
assert(creatinine_level_score(143,CREATININE_CONFIG_FILE)==3)
assert(creatinine_level_score(150,CREATININE_CONFIG_FILE)==4)
assert(creatinine_level_score(174,CREATININE_CONFIG_FILE)==5)
assert(creatinine_level_score(220,CREATININE_CONFIG_FILE)==6)
assert(creatinine_level_score(290,CREATININE_CONFIG_FILE)==8)

## #2D BMI

**NOTE:** You can assume that BMI will be a positive integer that is not more than 100

In [14]:
import json

def bmi_score(bmi, config_file):
    """ int, str -> int
    bmi: The BMI measurement as an integer value between 0 and 100
    config_file: The name of a configuration file to use for scoring the BMI
    
    The function should return a score for this subject based on where their BMI
    falls in the scoring rules.
    """
    
    score = 0
    
    ### BEGIN SOLUTION
    
    config = json.load(open(config_file))
    bmi_rules = config.get("bmi")
    
    for rule in bmi_rules:
        if bmi >= rule.get('min') and bmi < rule.get('max'):
            score = rule.get('points')
    
    ### END SOLUTION

    return score


In [15]:
# Put the name of your configuration file below
BMI_CONFIG_FILE = "maggic.json"

In [16]:
assert(bmi_score(10,BMI_CONFIG_FILE)==6)
assert(bmi_score(15,BMI_CONFIG_FILE)==5)
assert(bmi_score(23,BMI_CONFIG_FILE)==3)
assert(bmi_score(29,BMI_CONFIG_FILE)==2)
assert(bmi_score(32,BMI_CONFIG_FILE)==0)
assert(bmi_score(35,BMI_CONFIG_FILE)==0)

## #2F Ejection Fraction and Systolic

In [17]:
import json

def ef_bp(ef, bp, config_file):
    """ int, int, str -> int
    ef: The ejection fraction for this subject
    bp: The systollic BP for this subject
    config_file: The name of a configuration file to use for scoring the ef/bp combination

    You'll note in the MAGGIC rules that this score depends on both the ejection fraction range
    and the systollic BP range.  Your code will need to the combination of 
    values to find the right score.  There are three ejection fraction ranges
    and six systollic BP ranges.
    """
    
    score = 0
    
    ### BEGIN SOLUTION
    
    config = json.load(open(config_file))
    rules = config.get("ef_bp_and_age")
    bp_rules = {}
    
    for ef_rule in rules:
        if ef >= ef_rule.get('min') and ef < ef_rule.get('max'):
            bp_rules = ef_rule.get('bp')

    for bp_rule in bp_rules:
        if bp >= bp_rule.get('min') and bp < bp_rule.get('max'):
            score = bp_rule.get('points')
    
    ### END SOLUTION

    return score


In [18]:
# Put the name of your configuration file below
EF_BP_CONFIG_FILE = "maggic.json"

In [19]:
assert(ef_bp(20,90,EF_BP_CONFIG_FILE)==5)
assert(ef_bp(25,115,EF_BP_CONFIG_FILE)==4)
assert(ef_bp(10,133,EF_BP_CONFIG_FILE)==2)
assert(ef_bp(29,142,EF_BP_CONFIG_FILE)==1)
assert(ef_bp(20,160,EF_BP_CONFIG_FILE)==0)
assert(ef_bp(30,90,EF_BP_CONFIG_FILE)==3)
assert(ef_bp(35,115,EF_BP_CONFIG_FILE)==2)
assert(ef_bp(33,133,EF_BP_CONFIG_FILE)==1)
assert(ef_bp(39,142,EF_BP_CONFIG_FILE)==0)
assert(ef_bp(30,160,EF_BP_CONFIG_FILE)==0)
assert(ef_bp(40,90,EF_BP_CONFIG_FILE)==2)
assert(ef_bp(45,115,EF_BP_CONFIG_FILE)==1)
assert(ef_bp(43,133,EF_BP_CONFIG_FILE)==0)
assert(ef_bp(49,142,EF_BP_CONFIG_FILE)==0)
assert(ef_bp(40,160,EF_BP_CONFIG_FILE)==0)


## #2G Ejection Fraction and Age

In [20]:
import json

def ef_age(ef, age, config_file):
    """ int, int, str -> int
    ef: The ejection fraction for this subject
    age: The age for this subject
    config_file: The name of a configuration file to use for scoring the ef/age combination

    You'll note in the MAGGIC rules that this score depends on both the ejection fraction range
    and the subject's age.  Your code will need to the combination of 
    values to find the right score.  There are three ejection fraction ranges
    and seven age ranges.
    """
    
    score = 0
    
    ### BEGIN SOLUTION
    
    config = json.load(open(config_file))
    rules = config.get("ef_bp_and_age")
    age_rules = {}
    
    for ef_rule in rules:
        if ef >= ef_rule.get('min') and ef < ef_rule.get('max'):
            age_rules = ef_rule.get('age')

    for age_rule in age_rules:
        if age >= age_rule.get('min') and age < age_rule.get('max'):
            score = age_rule.get('points')
    
    ### END SOLUTION

    return score


In [21]:
# Put the name of your configuration file below
EF_AGE_CONFIG_FILE = "maggic.json"

In [22]:
assert(ef_age(20,30,EF_AGE_CONFIG_FILE)==0)
assert(ef_age(25,45,EF_AGE_CONFIG_FILE)==0)
assert(ef_age(21,57,EF_AGE_CONFIG_FILE)==1)
assert(ef_age(29,62,EF_AGE_CONFIG_FILE)==2)
assert(ef_age(20,67,EF_AGE_CONFIG_FILE)==4)
assert(ef_age(20,73,EF_AGE_CONFIG_FILE)==6)
assert(ef_age(20,75,EF_AGE_CONFIG_FILE)==8)
assert(ef_age(20,80,EF_AGE_CONFIG_FILE)==10)

assert(ef_age(30,30,EF_AGE_CONFIG_FILE)==0)
assert(ef_age(35,45,EF_AGE_CONFIG_FILE)==0)
assert(ef_age(31,57,EF_AGE_CONFIG_FILE)==2)
assert(ef_age(39,62,EF_AGE_CONFIG_FILE)==4)
assert(ef_age(30,67,EF_AGE_CONFIG_FILE)==6)
assert(ef_age(30,73,EF_AGE_CONFIG_FILE)==8)
assert(ef_age(30,75,EF_AGE_CONFIG_FILE)==10)
assert(ef_age(30,80,EF_AGE_CONFIG_FILE)==13)

assert(ef_age(40,30,EF_AGE_CONFIG_FILE)==0)
assert(ef_age(45,45,EF_AGE_CONFIG_FILE)==0)
assert(ef_age(41,57,EF_AGE_CONFIG_FILE)==3)
assert(ef_age(49,62,EF_AGE_CONFIG_FILE)==5)
assert(ef_age(40,67,EF_AGE_CONFIG_FILE)==7)
assert(ef_age(40,73,EF_AGE_CONFIG_FILE)==9)
assert(ef_age(40,75,EF_AGE_CONFIG_FILE)==12)
assert(ef_age(40,80,EF_AGE_CONFIG_FILE)==15)


# #3 MAGGIC_Score() Function

Now, we need to write a MAGGIC Score function that can calculate the score using the JSON configured rules you built in steps 1 and 2, given values for each of the following parameters:
* risk_factors - list of strings like "Male" and "Smoker"
* ejection_fraction - numeric value of ejection fraction
* nyha_class - the NYHA value
* creatinine - the creatinine level value
* bmi - the patient BMI
* systolic_bp - patient's systolic blood pressure
* age - the patient's age


In [23]:
def maggic_score(risk_factors, ejection_fraction, nhya_class, creatinine, bmi, systolic_bp, age):
    score = 0
    
    # NOTE: this function definition doesn't take the config_file name as one of the parameters.
    # I left it this way because it was up to you to decide if you wanted one config file
    # or one config file per set of rules.  Put the configuraiton file name(s) into your 
    # code below as needed.
    
    # BEGIN SOLUTION
    
    config_file = "maggic.json"
    score += risk_factors_score(risk_factors, config_file)
    score += ejection_fraction_score(ejection_fraction, config_file)
    score += nhya_class_score(nhya_class, config_file)
    score += creatinine_level_score(creatinine, config_file)
    score += bmi_score(bmi, config_file)
    score += ef_bp(ejection_fraction, systolic_bp, config_file)
    score += ef_age(ejection_fraction, age, config_file)

#     print(risk_factors_score(risk_factors, config_file))
#     print(ejection_fraction_score(ejection_fraction, config_file))
#     print(nhya_class, nhya_class_score(nhya_class, config_file))
#     print(creatinine_level_score(creatinine, config_file))
#     print(bmi_score(bmi, config_file))
#     print(ef_bp(ejection_fraction, systolic_bp, config_file))
#     print(ef_age(ejection_fraction, age, config_file))

    
    # END SOLUTION
    
    return score

In [24]:
import requests
import json

url = 'https://hds5210-2020.s3.amazonaws.com/test_subjects.json'

rsp = requests.get(url)
subjects = json.loads(rsp.text)

In [25]:
import json
import requests

# I've provided a test file at https://hds5210-2020.s3.amazonaws.com/test_subjects.json
# I'll be using the test cases in there for grading this final function
# If your tests above all worked for the provided test cases, then this should work fine, too.
# There's only one test case provided right now.  I'll add many more for the final scoring.


for subject in subjects:
    rf = subject.get('risks')
    ef = subject.get('ef')
    nhya = subject.get('nhya')
    creatinine = subject.get('creatinine')
    bmi = subject.get('bmi')
    bp = subject.get('bp')
    age = subject.get('age')
    maggic = subject.get('maggic')
    score = maggic_score(rf, ef, nhya, creatinine, bmi, bp, age)
    print("Got a score of {}, should have been {}".format(score, maggic))
    assert(score == maggic)

Got a score of 20, should have been 20
